## **CHAPTER 5 오차역전파법**

**앞에선 신경망의 가중치 매개변수의 기울기(정확히는 가중치 매개변수에 대한 손실 함수의 기울기)는 수치미분을 사용해 구했다.**

> - **수치 미분은 단순하고 구현하기도 쉽지만 꼐산 시간이 오래 걸린다는 게 단점이다.**

**이번 장에서는 가중치 매개변수의 기울기를 효율적으로 계산하는 '오차역전파법(backpropagation)'을 배워본다.**

**오차역전파법을 제대로 이해하는 방법은 두 가지가 있다.**

> - **하나는 수식을 통한 것이고, 다른 하나는 계산 그래프를 통한 것이다.**

**전자 쪽이 일반적인 방법으로, 특히 기계학습을 다루는 책 대부분은 수식을 중심으로 이야기를 전개한다.** 

> - **확실히 수식을 사용한 설명은 정확하고 간결하므로 올바른 방법이라 할 수 있다. 하지만 너무 오랜만에 수식을 중심으로 생각하다 보면 본질을 놓치거나, 수많은 수식에 당황하는 일이 벌어지기도 한다.**

**그래서 이번 장에서는 계산 그래프를 사용해서 '시각적'으로 이해를 한다.**

## **5.1 계산 그래프**

**계산 그래프(computational graph)는 계산 과정을 그래프로 나타낸 것이다. 여기에서의 그래프는 우리가 잘 아는 그래프 자료구조로, 복수의 노드(node)와 에지(edge)로 표현된다.**

### **5.1.1 계산 그래프로 풀다.**

**계산 그래프는 계산 과정을 노드와 화살표로 표현한다.**

> - **노드는 원(o)으로 표기하고 원 안에 연산 내용을 적는다. 또, 계산 결과를 화살표 위에 적어 각 노드의 계산 결과가 왼쪽에서 오른쪽으로 전해지게 한다.**

**문제1: 현빈군은 슈퍼에서 1개에 100원인 사과를 2개 샀다. 이때 지불 금액을 구하라. 단 소비세가 10% 부과된다.**

![test](./img/계산1.png)

**위 그림과 같이 처음에 사과의 100원이 'x2'노드로 흐르고, 200원이 되어 다음 노드로 전달된다. 이제 200원이 'x1.1' 노드를 거쳐 220원이 된다. 따라서 이 계산 그래프에 따르면 최종 답은 220원이다.**

![test](./img/계산2.png)

> - **'사과의 개수'와 '소비세'를 변수로 취급해 원 밖에 표기**

**문제 2: 현빈 군은 슈퍼에서 사과를 2개, 귤을 3개 샀다. 사과는 1개에 100원, 귤은 1개 150원이다. 소비세가 10%일 때 지불 금액을 구하라**

![test](./img/계산3.png)

**이 문제에는 덧셈 노드인 '+'가 새로 등장하여 사과와 귤의 금액을 합산한다. 계산 그래프는 왼쪽에서 오른쪽으로 계산을 진행한다. 회로에 전류가 흐르듯 계산 결과가 왼쪽에서 오른쪽으로 전달된다고 생각하면 된다.**

**지금까지 살펴본 것처럼 계산 그래프를 이용한 문제풀이는 다음 흐름으로 진행된다.**

> - **1.계산그래프를 구성한다.**

> - **2.그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.**

**여기서 2번째 "계산을 왼쪽에서 오른쪽으로 진행"하는 단계를 순전파(forward propagation)라고 한다. 순전파는 계산 그래프의 출발점부터 종착점으로의 전파이다.**

**순전파라는 이름이 있다면 반대 방향(그림에서 말하면 오른쪽에서 왼쪽)의 전파도 가능할까? 그것을 역전파(backward propagation)라고 한다. 역전파는 이후에 미분을 계산할 때 중요한 역할을 한다.**

### **5.1.2 국소적 계산**

**계산 그래프의 특징은 '국소적 계산'을 전파함으로써 최종 결과를 얻는다는 점에 있다. 국소적이란 '자신과 직접 관계된 작은 범위'라는 뜻이다.**

> - **국소적 계산은 결국 전체에서 어떤 일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다는 것이다.**

**가령 슈퍼마켓에서 사과 2개를 포함한 여러 식품을 구입하는 경우를 생각해보자.**

![test](./img/계산4.png)

**위 그림처럼 여러 식품을 구입하여 (복잡한 계산을 거쳐) 총 금액이 4,000원이 되었다. 여기에서 핵심은 각 노드에서의 계산은 국소적 계산이라는 점이다.**

> - **가령 사과와 그 외의 물품 값을 더하는 계산(4,000 + 200 -> 4,200)은 4,000이라는 숫자가 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 뜻이다. 각 노드는 자신과 관련한 계산 (이 예에서는 입력된 두 숫자의 덧셈) 외에는 아무것도 신경 쓸 게 없다.**

**이처럼 계산 그래프는 국소적 계산에 집중한다. 전체 계산이 제아무리 복잡하더라도 각 단계에서 하는 일은 해당 노드의 '국소적 계산'이다. 국소적인 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 해낸다.**

### **5.1.3 왜 계산 그래프로 푸는가?**

**'국소적 계산' 덕분에 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화한다. 또 다른 이점으로, 계산 그래프는 중간 계산 결과를 모두 보관할 수 있다.**

**실제 계산 그래프를 사용하는 가장 큰 이유는 역전파를 통해 '미분'을 효율적으로 계산할 수 있다는 점이다.**

> - **사과가격이 오르면 최종금액에 어떤 영향을 끼치는지 알고 싶다면, 이는 '사과 가격에 대한 지불 금액의 미분'을 구하는 문제에 해당한다. 이 미분 값은 사과 값이 '아주 조금' 올랐을 때 지불금액이 얼마나 증가하느냐를 표시한다.**

**'사과 가격에 대한 지불 금액의 미분' 같은 값은 계산 그래프에서 역전파를 하면 구할 수 있다.**


![test](./img/계산5.png)

**위 그림과 같이 역전파는 순전파와는 반대 방향으로 화살표(굵은 선)로 그린다. 이 전파는 '국소적 미분'을 전달하고 그 미분 값은 화살표 아래에 적는다.**

> - **이 예에서 역전파는 오른쪽에서 왼쪽으로 '1 -> 1.1 -> 2.2' 순으로 미분 값을 전달한다. 이 결과로부터 '사과 가격에 대한 지불 금액의 미분' 값은 2.2라 할 수 있다. (정확히는 사과 값이 아주 조금 오르면 최종 금액은 그 아주 작은 값의 2.2배 만큼 오른다는 뜻이다.)**

**여기에서는 사과 가격에 대한 미분을 구했지만, '소비세에 대한 지불 금액의 미분'이나 '사과 개수에 대한 지불 금액의 미분'도 같은 순서로 구할 수 있다.**

> - **그리고 그떄는 중간까지 구한 미분 결과를 공유할 수 있어서 다수의 미분을 효율적으로 계산할 수 있다.**

**이처럼 계산 그래프의 이점은 순전파와 역전파를 활용해서 각 변수의 미분을 효율적으로 구할 수 있다는 것이다.**

## **5.2 연쇄법칙**

**그동안 해온 계산 그래프의 순전파는 계산 결과를 왼쪽에서 오른쪽으로 전달했다. 이 순서는 평소 하는 방식이니 자연스럽게 느껴졌을 것이다.**

**한편 역전파는 '국소적인 미분'을 순방향과는 반대인 오른쪽에서 왼쪽으로 전달한다.(처음 보면 당황할지도 모른다.) 또한, 이 '국소적 미분'을 전달하는 원리는 연쇄법칙(chain rule)에 따른 것이다.**

### **5.2.1 계산 그래프의 역전파**

**y = f(x)라는 계산의 역전파**

![test](./img/연쇄1.png)

**위 그림과 같이 역전파의 계산 전파는 신호 E에 노드의 국소적 미분(∂y/∂x)를 곱한 후 다음 노드로 전달한다. 여기서 말하는 국소적 미분은 순전파 때의 y = f(x) 계산의 미분을 구한다는 것이며, 이는 x에 대한 y의 미분(∂y/∂x)을 구한다는 뜻이다.**

> - **가령 y = f(x) = x^2이라면 (∂y/∂x) = 2x가 된다. 그릭 이 국소적인 미분을 상류에서 전달된 값(이 예에서는 E)에 곱해 앞쪽 노드로 전달한다.**

**이것이 역전파의 계산 순서인데, 이러한 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다는 것이 이 전파의 핵심이다.**

### **5.2.2 연쇄법칙이란?**

**연쇄법칙을 설명하려면 우선 합성 함수 이야기부터 시작해야 한다. 합성 함수란 여러 함수로 구성된 함수이다.**

![test](./img/합성.png)

**예를 들어 z = (x+y)^2이라는 식은 아래 처럼 두 개의 식으로 구성된다.**

![test](./img/연쇄2.png)

**연쇄법칙은 합성 함수의 미분에 대한 성질이며, 다음과 같이 정의된다.**

> - **합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.**

### **5.2.3 연쇄법칙과 계산 그래프**

**연쇄법칙 계산을 계산 그래프로 그리면**

![test](./img/연쇄3.png)

**이고, 각각의 미분 결과값을 대입하면**

![test](./img/연쇄4.png)

**위 그림과 같이 계산 그래프의 역전파는 오른쪽에서 왼쪽으로 신호를 전파한다. 역전파의 계산 절차에서는 노드로 들어온 입력 신호에 그 노드의 국소적 미분(편미분)을 곱한 후 다음 노드로 전달한다.**

**주목할 것은 맨 왼쪽의 역전파로, 이 계산은 연쇄법칙에 따르면 (∂z/∂z)(∂z/∂t)(∂t/∂x) = (∂z/∂t)(∂t/∂x) = (∂z/∂x)가 성립되어 'x에 대한 z의 미분'이 된다. 즉, 역전파가 하는 일은 연쇄법칙의 원리와 같다는 것이다.**

## **5.3 역전파**

**앞 절에서는 계산 그래프의 역전파가 연쇄법칙에 따라 진행되는 모습을 설명했다. 이번 절에서는 '+'와 'x'등의 연산을 예로 들어 역전파의 구조를 설명한다.**

**z = x + y라는 식을 대상으로 덧셈 노드의 역전파를 살펴본다.**

![test](./img/연쇄5.png)

**위 식과 같이 (∂z/∂x) = 1, (∂z/∂y) = 1이 된다. 이를 계산 그래프로 그려보면**

![test](./img/연쇄6.png)

**덧셈노드의 역전파는 1을 곱하기만 할 뿐이므로 입력된 값을 그대로 다음 노드에 보내게 된다.**

> - **이 예에서는 상류에서 전해진 미분 값을 (∂L/∂z)이라 했는데, 최종적으로 L이라는 값을 출력하는 큰 계산 그래프를 가정하기 때문이다. z = x + y 계산은 그 큰 계산 그래프의 중간 어디가에 존재하고, 상류로부터 ∂L/∂z 값이 전해진 것이다. 그리고 다시 하류로는 ∂L/∂x과 ∂L/∂y 값을 전달한다.**

**구체적인 예로 가령 '10+5=15'라는 계산이 있고, 상류에서 1.3이라는 값이 흘러오면, 이를 계산 그래프로 그리면**

![test](./img/연쇄7.png)

**덧셈 노드 역전파는 입력 신호를 다음 노드로 출력할 뿐이므로 위 그림 처럼 1.3을 그대로 다음 노드로 전달한다.**

### **5.3.2 곱셈 노드의 역전파**

**이어서 곱셈 노드의 역전파를 생각해본다. z = xy라는 식을 생각해보면, 이 식의 미분은 다음과 같다.**

![test](./img/연쇄8.png)

**위 식에서 계산 그래프는 다음과 같이 나타낼 수 있다.**

![test](./img/연쇄9.png)

**곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을  곱해서 하류로 보낸다.**

> - **서로 바꾼 값이란 위 그림처럼 순전파 때 x였다면 역전파에서는 y, 순전파 때 y였다면 역전파에서 x로 바꾼다는 의미이다.**

**가령 '10 x 5 = 50'이라는 계산이 있고, 역전파 때 상류에서 1.3 값이 흘러나온다고 했을 때 이를 계산 그래프로 그리면**

![test](./img/연쇄10.png)

**곱셈의 역전파에서는 입력 신호를 바꾼 값을 곱하여 하나는 1.3 x 5 = 6.5, 다른 하나는 1.3 x 10 = 13이 된다. 덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값은 필요하지 않습니다만, 곱셈의 역전파는 순방향 입력 신호의 값이 필요하다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둔다.**

![test](./img/연쇄11.png)

**사과 가격에 대한 미분은 2.2, 사과개수에 대한 미분은 110, 소비세에 대한 미분은 650이다. 이는 소비세와 사과 가격이 같은 양만큼 오르면 최종금액에는 소비세가 650의 크기로, 사과가격이 2.2 크기로 영향을 줄 수 있다고 할 수 있다. 단 이 예에서는 소비세와 사과 가격은 단위가 다르다.(소비세 1 100%, 사과가격 1은 1원)**

> - **1만큼 오르면 사과가격이 2.2원 소비세 6.5원이 최종금액에 영향을 준다.**

## 5.4 단순한 계층 구현하기
- 계산 그래프의 곱셈 노드를 MulLayer, 덧셈 노드를 AddLayer라는 이름으로 구현한다.

## 5.4.1 곱셈 계층
- 모든 계층은 forward()와 backward()라는 공통의 메서드를 갖도록 구현할 것이다.
![test](./img/init.png)

In [5]:
class MulLayer:
    def __init__(self):
        self.x = None # 변수 초기화
        self.y = None 
    def forward(self,x,y): # 순전파 
        self.x = x # 인수
        self.y = y # 인수
        out = x * y
        
        return out
    
    def backward(self,dout): # 역전파
        dx = dout * self.y # 곱셈 노드의 역전파에 따라 x,y를 바꾼다.
        dy = dout * self.x
        
        return dx,dy

- init()에서는 인스턴스 변수인 x와 y를 초기화한다. 이 두 변수는 순전파 시의 입력 값을 유지하기 위해서 사용
- forward() 에서는 x와 y를 인수로 받고 두 값을 곱해서 반환한다.
- 반면 backward()에서는 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘린다.

In [6]:
apple = 100 # 사과
apple_num = 2 # 사과 갯수
tax = 1.1 # 소비세

In [7]:
# 계층들
mul_apple_layer = MulLayer() # 변수 초기화 __init__
mul_tax_layer = MulLayer()

In [8]:
apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price,tax)
print(price)

220.00000000000003


In [9]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple,dapple_num,dtax)

2.2 110.00000000000001 200


## 5.4.2 덧셈 계층
- 덧셈 계층에서는 초기화가 필요 없으니 init() 에서는 아무 일도 하지 않는다.(pass가 아무것도 하지 말라는 명령이다.) 

In [10]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y): # 입력 받은 두 인수 x,y를 더해서 반환
        out = x+y
        return out
    def backward(self,dout): # 상류에서 내려온 미분을 그대로 하류로 흘린다.
        dx = dout*1
        dy = dout*1
        return dx,dy

In [11]:
apple = 100
orange = 150
apple_num = 2
orange_num = 3
tax = 1.1
dprice = 1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
Add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple,apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
apor_price = Add_apple_orange_layer.forward(apple_price,orange_price)
price = mul_tax_layer.forward(apor_price,tax)


# 역전파
dapor_price, dtax = mul_tax_layer.backward(dprice)
dapple_price,dorange_price = Add_apple_orange_layer.backward(dapor_price)
dapple,dapple_num = mul_apple_layer.backward(dapple_price)
dorange,dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple_num,dapple,dorange,dorange_num,dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기
- 계산 그래프를 신경망에 적용
> - 신경망을 구성하는 층(계층) 각각을 클래스 하나로 구현한다.
> - 우선은 활성화 함수인 ReLU와 Sigmoid 계층을 구현한다.

## 5.5.1 ReLU 계층
![test](./img/렐루식.png)
- 위 식에서 x에 대한 y의 미분은 x가 0 보다 클때는 1 x가 0 보다 같거나 작을때는 0이다.
![test](./img/렐루계산그래프.png)
- 위 그림과 같이 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 그대로 하류로 흘린다.
- 반면, 순전파 때의 x가 0 이하면 역전파 때는 하류로 신호를 보내지 않는다.(0을 보낸다.)

In [12]:
class Relu:
    def __init__(self):
        self.mask = None # Relu 클래스는 mask라는 인스턴스 변수를 가진다.
        
    def forward(self,x):
        self.mask = (x<=0) # x가 0 보다 작으면 True
        out = x.copy() # 인수를 copy해서 out변수에 할당
        out[self.mask] = 0 # out[self.mask] mask의 원소가 True인 곳에 0을 할당
        
        return out
    
    def backward(self,dout):
        dout[self.mask] = 0 # dout인수의 True값에 0을 할당
        dx = dout 
        # 순전파 때의 입력값이 0이하면 역전파 때의 값은 0이 되어야한다.
        # 그래서 역전파 때는 순전파 때 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서
        # 전파된 dout을 0으로 설정
        return dx

In [13]:
import numpy as np
x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)
mask = (x<=0)
print(mask)
x[mask] = 0
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]
[[1. 0.]
 [0. 3.]]


## 5.5.2 Sigmoid 계층
![test](./img/식그모이드.png)

![test](./img/시그계산.png)

- 간소화 버전은 역전파 과정의 중간 계산들을 생략할 수 있어 더 효율적인 계산이라 말할 수 있다.
- 또한, 노드를 그룹화하여 Sigmoid 계층의 세세한 내용을 노출하지 않고 입력과 출력에만 집중 할 수 있다는 것도 중요한 포인트이다.

In [14]:
class Sigmoid:
    def __init__(self): # 순전파 출력을 인스턴스 변수 out에 보관했다가, 역전파 계산 때 그 값을 사용
        self.out = None
    def forward(self,x):
        out = 1 / (1+np.exp(-x))
        self.out = out
    def backward(self,dout):
        dx = dout * (1.0 - dout) * self.out
        
        return dx

## 5.6.1 Affine 계층
- 신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서는 어파인 변환(affine transformation)이라 한다. 그래서 이 책에서는 어파인 변환을 수행하는 처리를 Affine계층이라 한다.

![test](./img/어파인.png)

- np.dot(X,W) + B 계산 그래프
- X,W,B가 행렬(다차원 배열)이라는 점에 주의

![test](./img/행역전파.png)

- 행렬을 사용한 역전파

![test](./img/전치.png)

![test](./img/어파인역전파.png)

- 각 변수의 형상에 주의해야한다.
> - 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야 하는데, 이를 위해서는 X에 대한 L미분과 W에 대한 L 미분식이 동원해야 할 수도 있기 때문이다.
>> - 예를 들어 Y에 대한 L 미분의 형상이 (3,)이고 W의 형상이(2,3)일 때, X에 대한 L미분의 형상이 (2,)가 되는 Y에대한 L미분과 W의 곱을 생각해보면 위 식이 유도된다.

![test](./img/어파인곱.png)

- 행렬 곱의 역전파는 행렬의 대응하는 차원의 원소 수가 일치하도록 곱을 조립하여 구할 수 있다.

## 5.6.2 배치용 Affine 계층

![test](./img/배치어파인.png)

- 데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층을 생각해보자
> - 묶은 데이터를 '배치'라고 부른다.
- 편향을 더할 때도 주의해야 한다. 순전파 때의 편향 덧셈은 XW에 대한 편향이 각 데이터에 더해진다.
> - 예를 들어 N=2(데이터가 2개)로 한 경우, 편향은 그 두 데이터 각각에 더해진다.
> - 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구한다.

In [15]:
X_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(X_dot_W)

[[ 0  0  0]
 [10 10 10]]


In [16]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [17]:
dY = np.array([[1,2,3],[4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [18]:
dB = np.sum(dY,axis=0) # 열을 기준으로 합
dB

array([5, 7, 9])

In [53]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        
        return dx

## 5.6.3 Softmax-with-Loss 계층
- 앞서 말했듯이 소프트맥스 함수는 입력 값을 정규화하여 출력한다.
![test](./img/soft.png)
> - 입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환되고, 마지막 Softmax 계층에 의해서 10개의 입력이 정규화된다. 이 그림에서는 숫자 '0'의 점수는 5.3이며, 이것이 Softmax 계층에 의해서 0.008(0.8%)로 변환된다. 
- Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력
> - 신경망에서 수행하는 작업은 학습과 추론 두 가지이다. 추론할 때는 일반적으로 소프트맥스 계층을 사용하지 않는다. 즉, 마지막 Affine 계층의 출력을 인식 결과로 이용한다 
> - 또한, 신경망에서 정규화하지 않은 출력 결과에서는 Softmax 앞의 Affine 계를 점수라 한다.
> - 즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면되니 Softmax 계층은 필요없다.
> - 반면 신경망을 학습할 때는 Softmax 계층이 필요하다.

![test](./img/소프트계산그래프.png)

![test](./img/간소화1.png)

- 위 그림에서는 3클래스 분류를 가정하고 이전 계층에서 3개의 입력(점수)을 받는다.
- 그림과 같이 Softmax 계층은 입력(a1,a2,a3)를 정규화하여 (y1,y2,y3)를 출력한다.
- Cross Entropy Error 계층은 Softmax의 출력 (y1,y2,y3)와 정답 레이블 (t1,t2,t3)를 받고, 이들 데이터로부터 손실 L을 출력
- 여기서 주목할 것은 역전파의 결과이다. 
> - Softmax 계층의 역전파는 (y1-t1,y2-t2,y3-t3)라는 말끔한 결과를 내놓고 있다.
> - 이는 Softmax의 출력과 정답 레이블의 차분이다.
> - 신경망의 역전파에서는 이 차이인 오차가 앞 계층에 전해지는 것이다.
- 신경망 학습의 목적은 신경망의 출력(소프트맥스출력)이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정하는 것이었다. 그래서 신경망의 출력과 정답 레이블의 오차를 효율적으로 앞 계층에 전달해야 한다.
- 앞의 역전파 결과는 바로 소프트 맥스 계층의 출력과 정답 레이블의 차이로, 신경망의 현재 출력과 정답 레이블의 오차를 있는 그대로 드러내는 것이다.
> - 소프트맥스 함수의 손실 함수로 교차 엔트로피 오차를 사용하니 역전파가 말끔하게 떨어진다.

In [54]:
def sigmoid(x):
    return 1/(1+np.exp(-x)) 
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) # 분자 분모 양쪽에 입력값의 최대 값을 빼준다. 결과는 같다.
    sum_exp_a = np.sum(exp_a) # 오버플로 대책
    y = exp_a / sum_exp_a
    return y
def cross_entropy_error(y,t):
    delta = 1e-7 # log0이 되는 것을 방지
    return -np.sum(t*np.log(y+delta))
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [55]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None
        self.t = None
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        # 역전파 때는 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층에 전파
        return dx
        

## 5.7 오차역전파법 구현하기

## 5.7.1 신경망 학습의 전체 그림
- 전체
> - 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라 한다.
- 1단계
> - 훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표
- 2단계
> - 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시
- 3단계
> - 가중치 매개변수를 기울기 방향으로 아주 조금 갱신
- 4단계
> - 위 과정을 반복한다.

In [56]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

In [57]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [58]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

In [59]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_backprop = network.gradient(x_batch, t_batch)



In [68]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.129 0.1329
0.0993 0.1032
0.09871666666666666 0.098
0.09871666666666666 0.098


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in subtract
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less_equal
  


0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
0.09871666666666666 0.098
